In [52]:
from numpy.random import seed
seed(1)
import sklearn
sklearn.__version__
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

import re
from sklearn.metrics import classification_report

In [53]:
data = pd.read_csv('./data/labeled_weather_data.csv', sep="\t")
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23239 entries, 0 to 23238
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   time         23239 non-null  object 
 1   month        23239 non-null  int64  
 2   temperature  23239 non-null  int64  
 3   feelslike    23239 non-null  int64  
 4   wind         23239 non-null  int64  
 5   direction    23239 non-null  object 
 6   gust         23239 non-null  int64  
 7   cloud        23239 non-null  int64  
 8   rain         23239 non-null  float64
 9   rain_ratio   23239 non-null  int64  
 10  pressure     23239 non-null  int64  
 11  weather      23239 non-null  object 
 12  label        23239 non-null  object 
dtypes: float64(1), int64(8), object(4)
memory usage: 2.3+ MB


In [54]:
# Xếp loại các loại thời tiết thành 2 dạng là có mưa và không mưa
list_norain = ['Clear', 'Fog', 'Cloudy', 'Mist', 'Partly cloudy', 'Thundery outbreaks possible']
list_rain = ['Light drizzle','Light rain','Light rain shower', 'Patchy light drizzle', 'Patchy light rain', 
             'Patchy light rain with thunder','Patchy rain possible','Heavy rain','Heavy rain at times',
             'Moderate or heavy rain shower','Moderate rain', 'Moderate rain at times', 'Overcast']

In [33]:
# Lấy ra cột y để train
# Cột y này là giá trị thời tiết được xếp loại
y = []
for w in data.weather:
    if w in list_norain:
        y+= ['norain']
    elif w in list_rain:
        y+= ['rain']
    else:
        y+= ['unknown']
# Do input x có giá trị dự đoán là giá trị thời tiết của 3 giờ tiếp theo
# nên chúng ta xóa dòng đầu tiên
y.pop(0)
print(y)

['rain', 'rain', 'rain', 'norain', 'norain', 'norain', 'norain', 'norain', 'rain', 'rain', 'norain', 'rain', 'norain', 'norain', 'norain', 'rain', 'rain', 'rain', 'rain', 'rain', 'rain', 'norain', 'rain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'rain', 'rain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'rain', 'rain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'rain', 'norain', 'norain', 'norain', 'norain', 'rain', 'norain', 'rain', 'norain', 'rain', 'rain', 'norain', 'rain', 'rain', 'rain', 'rain', 'norain', 'rain', 'norain', 'norain', 'norain', 'rain', 'norain', 'rain', 'rain', 'norain', 'norain', 'norain', 'norain', 'rain', 'rain', 'rain', 'rain', 'rain', 'rain', 'rain', 'rain', 'rain', 'rain', 'rain', 'rain', 'rain', 'rain', 'rain', 'rain', 'rain', 'rain', 'rain', 'rain', 'norain', 'norain', 'norain',

In [37]:
# Do input x có giá trị dự đoán là giá trị thời tiết của 3 giờ tiếp theo nhưng không có giá trị đó nên xóa dòng cuối cùng
x = data.drop(23238)


In [38]:
# Tách tập train và tập test với tỉ lệ 9-1
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, stratify = y, random_state=0)

In [39]:
# Tách tập train và tập validation với tỉ lệ 8-2
x_train_v, x_val, y_train_v, y_val = train_test_split(x_train, y_train, test_size = 0.2, stratify = y_train, random_state=0)

In [40]:
# Góm nhóm các giá trị của các cột có kiểu category
# Cột month gom thành 2 giá trị gồm "mua" (5->11) và "kho" (12->4)
# Cột direction thì xóa bỏ kí tự đầu tiên nếu vượt quá 2 ký tự
# Cột weather thì gom nhóm giống như khi gom nhóm y
class ColStd(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df,y=None):
        df = X_df[['time',
                   'month',
                   'temperature',
                   'feelslike',
                   'wind',
                   'direction',
                   'gust',
                   'cloud',
                   'rain',
                   'pressure',
                   'weather']]#khong lay cot precipitatinon
        temp_time =[]
        for x in df['time']:
            t = int(re.findall(r'(\d{1,2}):\d{2}',x)[0])
            if ((t>=6) & (t<=15)): 
                temp_time+=['sang']
            else:
                temp_time+=['toi']
        df.time=temp_time
        temp_month =[]
        for x in df['month']:
            if x in [5,6,7,8,9,10,11]:
                temp_month += ['mua'] 
            elif x in [12,1,2,3,4]:
                temp_month += ['kho']
            else:
                temp_month += ['kho']
        df.month = temp_month
        temp_dic = []
        for x in df['direction']:
            if len(x) == 3:
                temp_dic += [x[1:]]
            else:
                temp_dic += [x]
        df.direction = temp_dic
        temp_wea = []
        for w in df['weather']:
            if w in list_norain:
                temp_wea+= ['norain']
            #elif w in list_ltrain:
            #    temp_wea+= ['ltrain']
            elif w in list_rain:
                temp_wea+= ['rain']
            else:
                temp_wea+= ['unknown']
        df.weather = temp_wea
        return df

In [41]:
# Tạo các Pipeline để tiền xử lý các giá trị có trong dữ liệu
nume_cols = ['temperature', 'feelslike', 'wind', 'gust', 'cloud', 'rain', 'pressure']
#nume_cols_nan = ['temperature', 'feelslike', 'wind', 'gust', 'cloud', 'humidity','precipitation', 'pressure']
cate_cols = ['time', 'month', 'direction', 'weather']

nume_trans = Pipeline(steps=[
    ('imputer1', SimpleImputer(missing_values = np.nan, strategy = 'mean')),
    ('imputer2', SimpleImputer(missing_values = 0, strategy='mean'))
])

cate_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy= 'most_frequent')),
    ('onehot', OneHotEncoder(dtype=int, handle_unknown= 'ignore'))
])

cols_trans = ColumnTransformer(
    transformers=[
        ('nume', nume_trans, nume_cols),
        ('cate', cate_trans, cate_cols)
])

preprocess_pipeline = Pipeline(steps=[
    ('colstd', ColStd()),
    ('coltrans', cols_trans),
    ('std', StandardScaler())
])

# Tiền xử lý tập train x
preprocessed_train_X = preprocess_pipeline.fit_transform(x_train_v)

C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [42]:
# Tiền xử lý tập validation x
preprocess_val_X = preprocess_pipeline.transform(x_val)

C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [43]:
from sklearn.utils.class_weight import compute_class_weight 

In [45]:
#khảo sát số lượng từng nhãn xem có cân bằng hay không
rain = 0
norain =0
for y in y_train:
    if y =='rain':
        rain+=1
    else:
        norain+=1
print(f"rain: {rain} norain: {norain}" )

rain: 8867 norain: 12047


In [46]:
# Tạo ra full_pipeline bao gồm tiền xử lý và model
mlpclf_model = Pipeline(steps=[
    ('pre', preprocess_pipeline),
    ('mlpclf', MLPClassifier(hidden_layer_sizes=(20),activation='tanh', solver='lbfgs', random_state=0, max_iter=500,))
])
#hidden_layer_sizes = (10, 20, 50, 100)

In [47]:
# train và tìm ra các tham số tốt nhất
train_errs = []
val_errs = []
iter = []
alphas = [0.001,0.01, 0.1, 1, 10,20, 100]

hidden_layer_sizes = [1,10,20,50,70,100]
best_val_err = float('inf'); best_alpha = None; best_hls = None;
for alpha in alphas:
    for hls in hidden_layer_sizes:
        iter +=[f"alpha = {alpha}, hls = {hls}"]
        print(iter[-1])
        mlpclf_model.set_params(mlpclf__alpha=alpha,mlpclf__hidden_layer_sizes=(hls))
        mlpclf_model.fit(x_train_v,y_train_v)
        train_err =100*(1 - mlpclf_model.score(x_train_v,y_train_v))
        val_err =100*(1 - mlpclf_model.score(x_val,y_val))

        train_errs += [train_err]
        val_errs +=[val_err]
        if ((val_err < best_val_err)):
            best_val_err = val_err
            best_alpha = alpha
            best_hls = hls


alpha = 0.001, hls = 1


C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

alpha = 0.001, hls = 10


c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

alpha = 0.001, hls = 20


...

C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

alpha = 100, hls = 70


C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

alpha = 100, hls = 100


C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [48]:
for i in range(len(train_errs)):
    print(f"{iter[i]} \n train_err :{train_errs[i]}\t val_err:{val_errs[i]}")
    print()
print(f"best alpha: {best_alpha}")
print(f"best hidden layer size: {best_hls}")

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


alpha = 0.001, hls = 1 
 train_err :22.36566851951467	 val_err:21.92206550322735

alpha = 0.001, hls = 10 
 train_err :20.56661287430518	 val_err:22.280659813530956

alpha = 0.001, hls = 20 
 train_err :20.243858705397166	 val_err:22.591441549127424

alpha = 0.001, hls = 50 
 train_err :17.97262566493336	 val_err:23.308630169734645

alpha = 0.001, hls = 70 
 train_err :17.21355567509414	 val_err:23.954099928281135

alpha = 0.001, hls = 100 
 train_err :16.20345466499312	 val_err:25.555821180970597

alpha = 0.01, hls = 1 
 train_err :22.371645448568522	 val_err:21.89815921587378

alpha = 0.01, hls = 10 
 train_err :20.64431295200526	 val_err:22.208940951470236

alpha = 0.01, hls = 20 
 train_err :19.705935090550476	 val_err:22.256753526177388

alpha = 0.01, hls = 50 
 train_err :17.954694877771804	 val_err:23.260817595027493

alpha = 0.01, hls = 70 
 train_err :17.273324965632654	 val_err:24.049725077695438

alpha = 0.01, hls = 100 
 train_err :15.83288506365429	 val_err:25.292852020081

In [50]:
for i in range(len(train_errs)):
    if (((i+1)%6==0) & (i!=0)):
        print(f"{round(train_errs[i],7)}",end ="\n")
    else:
        print(f"{round(train_errs[i],7)}",end ="\t")

print()
for i in range(len(val_errs)):
    if (((i+1)%6==0 & (i!=0))):
        print(f"{round(val_errs[i],7)}",end ="\n")
    else:
        print(f"{round(val_errs[i],7)}",end ="\t")

22.3656685	20.5666129	20.2438587	17.9726257	17.2135557	16.2034547
22.3716454	20.644313	19.7059351	17.9546949	17.273325	15.8328851
22.3537147	20.6323591	19.885243	18.2415875	17.0103401	15.8747236
22.3118762	20.7160361	19.96892	18.534457	17.8710179	16.8071245
22.3477377	20.811667	20.5008667	19.8732891	20.1721356	19.8732891
22.3357839	20.9132748	20.7459207	20.7459207	20.72799	20.7817823
22.2521069	21.7141833	21.7440679	21.821768	21.7141833	21.8636065

21.9220655	22.2806598	22.5914415	23.3086302	23.9540999	25.5558212
21.8981592	22.208941	22.2567535	23.2608176	24.0497251	25.292852
21.8981592	22.2806598	21.8264404	23.213005	24.0019125	25.3167583
21.8981592	21.5634712	21.6351901	23.6194119	23.2608176	24.7430074
21.8503466	21.3244083	21.3244083	21.5395649	21.1092517	21.2048769
21.8025341	21.2287832	21.2526895	21.1809706	21.2287832	21.2526895
21.8264404	21.4200335	21.3961272	21.4917523	21.3961272	21.4917523


In [51]:
svmclf_model = Pipeline(steps=[
    ('pre', preprocess_pipeline),
    ('svm', SVC(kernel = 'rbf'))
])
c = [0.1,1,10, 20, 50,100]
gamma = [0.0001,0.001,0.01,0.1,1]
train_errs1 = []
val_errs1 = []
iter = []
best_val_err = float('inf'); best_C = None; best_gamma=None
for C in c:
    for g in gamma:
        iter +=[f"C={C}, g={g}"]
        print(iter[-1])
        svmclf_model.set_params(svm__C=C,svm__gamma=g)
        svmclf_model.fit(x_train_v,y_train_v)
        train_err =100*(1 - svmclf_model.score(x_train_v,y_train_v))
        val_err =100*(1 - svmclf_model.score(x_val,y_val))
        train_errs1 += [train_err]
        val_errs1 +=[val_err]
        if ((val_err < best_val_err)):
            best_val_err = val_err
            best_C = C
            best_gamma =g
    

C=0.1, g=0.0001


C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

C=0.1, g=0.001


C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

C=0.1, g=0.01


...

C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

C=100, g=0.1


C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

C=100, g=1


C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [56]:
for i in range(len(train_errs1)):
    print(f"{iter[i]} \n train_err :{train_errs1[i]}\t val_err:{val_errs1[i]}")
    print()
print(f"best C: {best_C}")
print(f"best gamma: {best_gamma}")

C=0.1, g=0.0001 
 train_err :42.25688841073456	 val_err:42.21850346641166

C=0.1, g=0.001 
 train_err :22.305899228976156	 val_err:21.611283767630884

C=0.1, g=0.01 
 train_err :22.305899228976156	 val_err:21.611283767630884

C=0.1, g=0.1 
 train_err :21.989121989121983	 val_err:21.39612718144872

C=0.1, g=1 
 train_err :25.372063833602297	 val_err:27.779105904852976

C=1, g=0.0001 
 train_err :22.305899228976156	 val_err:21.611283767630884

C=1, g=0.001 
 train_err :22.305899228976156	 val_err:21.611283767630884

C=1, g=0.01 
 train_err :22.305899228976156	 val_err:21.611283767630884

C=1, g=0.1 
 train_err :21.098559560098018	 val_err:21.276595744680847

C=1, g=1 
 train_err :15.498177036638571	 val_err:22.5197226870667

C=10, g=0.0001 
 train_err :22.305899228976156	 val_err:21.611283767630884

C=10, g=0.001 
 train_err :22.305899228976156	 val_err:21.611283767630884

C=10, g=0.01 
 train_err :22.24612993843763	 val_err:21.635190054984466

C=10, g=0.1 
 train_err :19.26364234056541	

In [58]:
for i in range(len(train_errs1)):
    if (((i+1)%5==0 & (i!=0))):
        print(f"{round(train_errs1[i],7)}",end ="\n")
    else:
        print(f"{round(train_errs1[i],7)}",end ="\t")
print()
for i in range(len(val_errs1)):
    if (((i+1)%5==0 & (i!=0))):
        print(f"{round(val_errs1[i],7)}",end ="\n")
    else:
        print(f"{round(val_errs1[i],7)}",end ="\t")

42.2568884	22.3058992	22.3058992	21.989122	25.3720638
22.3058992	22.3058992	22.3058992	21.0985596	15.498177
22.3058992	22.3058992	22.2461299	19.2636423	7.5189767
22.3058992	22.3058992	22.1923376	18.3790568	5.648198
22.3058992	22.3058992	22.0608451	17.2374634	3.7116729
22.3058992	22.3058992	21.9472835	16.3289702	2.8270874

42.2185035	21.6112838	21.6112838	21.3961272	27.7791059
21.6112838	21.6112838	21.6112838	21.2765957	22.5197227
21.6112838	21.6112838	21.6351901	21.2765957	27.2292613
21.6112838	21.6112838	21.6590963	21.9220655	28.4962945
21.6112838	21.6112838	21.5156586	22.0176907	30.6478604
21.6112838	21.6112838	21.4439398	22.376285	30.9108295


In [59]:
#điểm của mô hình mlpclf khi áp dụng tham số tốt nhất dự đoán trên tập tập x_val
mlpclf_model.set_params(mlpclf__alpha=best_alpha,mlpclf__hidden_layer_sizes=best_hls)
mlpclf_model.fit(x_train_v, y_train_v)
y_pred = mlpclf_model.predict(x_val)
print(classification_report(y_val, y_pred))

C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

              precision    recall  f1-score   support

      norain       0.81      0.83      0.82      2410
        rain       0.76      0.74      0.75      1773

    accuracy                           0.79      4183
   macro avg       0.78      0.78      0.78      4183
weighted avg       0.79      0.79      0.79      4183



c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [60]:
#điểm của mô hình svmclf khi áp dụng tham số tốt nhất dự đoán trên tập tập x_train_v
svmclf_model.set_params(svm__C=best_C,svm__gamma=best_gamma)
svmclf_model.fit(x_train_v,y_train_v)
y_pred = svmclf_model.predict(x_val)

print(classification_report(y_val, y_pred))

C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

              precision    recall  f1-score   support

      norain       0.81      0.82      0.82      2410
        rain       0.75      0.75      0.75      1773

    accuracy                           0.79      4183
   macro avg       0.78      0.78      0.78      4183
weighted avg       0.79      0.79      0.79      4183



# NHẬN XÉT
- 2 mô hình khá cân băng
- precision của MLP của nhãn rain cao hơn của MLP 0.1
- MLP tốt hơn SVM 1 chút trong việc dự đoán ra nhãn "rain" nên chọn MLP

In [61]:
#huấn luyện trên toàn tập train
mlpclf_model.set_params(mlpclf__alpha=best_alpha,mlpclf__hidden_layer_sizes=best_hls)
mlpclf_model.fit(x_train,y_train)
y_pred = svmclf_model.predict(x_train)

print(classification_report(y_train, y_pred))

C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

              precision    recall  f1-score   support

      norain       0.81      0.83      0.82     12047
        rain       0.76      0.74      0.75      8867

    accuracy                           0.79     20914
   macro avg       0.78      0.78      0.78     20914
weighted avg       0.79      0.79      0.79     20914



In [62]:
# Điểm mô hình trên test set
y_pred = mlpclf_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      norain       0.81      0.83      0.82      1339
        rain       0.76      0.73      0.74       985

    accuracy                           0.79      2324
   macro avg       0.78      0.78      0.78      2324
weighted avg       0.79      0.79      0.79      2324



C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_6652\1976044800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

# Nhận xét
- Độ chính xác của mô mình đạt được mong đợi (>80%)
- Mô hình dự đoán tốt đối với nhãn không mưa (f1 : 0.91)
- Mô hình dự đoán nhãn mưa không tốt lắm, nhưng cũng không hề tệ tuy nhiên độ hiệu quả cũng tương đối
    -Đưa ra được 65% trường hợp mưa với độ chính xác 78%

# Kết luận
- Dự đoán là khi áp dụng vào thực tế, mô hình hoạt động sẽ không tốt, nhưng không quá tệ
- Có thể sử dụng làm nguồn tham khảo nếu không thể xem dự báo thời tiết vì một lý do nào đó 